# Lab Three: Extending Logistic Regression

Team: Miro Ronac, Kirk Watson, Brandon Vincitore

## 1. Preparation and Overview

### 1.1 Task and Use-case

This data can be useful in identifying prediabetes or diabetes in patients and assisting doctors with making accurate observations from a variety of health indicators.

Every year, the CDC collects data from a health-related telephone survey called the Behavioral Risk Factor Surveillance System. The data gathered from these surveys include information on “health-related risk behaviors, chronic health conditions, and use of preventive services.” This dataset focuses on responses from 2015 and diabetes, a “prevalent chronic disease in the United States.”

Ultimately, the ability to identify a patient with prediabetes or diabetes with increased efficiency and accuracy is the intention of analyzing this dataset. With this capability, a diabetes diagnosis can be reached at a faster rate compared to when a human makes the diagnosis. Diabetes is extremely common in the US as about 1 in 10 Americans have diabetes, and [about 1 in 5 people with diabetes don’t know they have it](https://www.cdc.gov/diabetes/library/spotlights/diabetes-facts-stats.html#:~:text=37.3%20million%20Americans%E2%80%94about%201,t%20know%20they%20have%20it.). In addition, 1 in 3 Americans have prediabetes, and [more than 8 in 10 adults with prediabetes don’t know they have it](https://www.cdc.gov/diabetes/library/spotlights/diabetes-facts-stats.html#:~:text=37.3%20million%20Americans%E2%80%94about%201,t%20know%20they%20have%20it.). Using this classifier, patients that might be at risk of diabetes can be brought to a doctor’s attention at a higher rate allowing for earlier medical care and attention.

Dataset Source: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

### 1.2 Dataset Preparation

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')
print('Size of dataset:', df.shape[0])
df.head()

Size of dataset: 253680


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [16]:
# Creating target features by modifying first column in original dataframe such that we have 3 features consisting of binary values for
# no diabetes, prediabetes, and diabetes where 0 is False and 1 is True
target_array = np.zeros((len(df),4))
for i in range(len(df)):
    target_array[i,0] = df['Diabetes_012'].values[i]
for i in range(len(target_array)):
    # no diabetes
    if target_array[i,0] == 0:
        target_array[i,1] = 1
    # prediabetes
    if target_array[i,0] == 1:
        target_array[i,2] = 1
    # diabetes
    if target_array[i,0] == 2:
        target_array[i,3] = 1

# Adding new target columns to original dataframe
target_columns = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for i in range(target_array.shape[1]-1):
    df.insert(i, target_columns[i], target_array[:,1:][:,i], True)
df_target = df.drop('Diabetes_012', axis=1)

columns = list(df_target.columns)
for col in columns:
    df_target[col] = df_target[col].astype(int)
df_target.head()

df_target.drop(df_target.tail(252000).index,inplace = True)
print('Size of dataset:', df_target.shape[0])
df_target.head()

Size of dataset: 1680


,NoDiabetes,PreDiabetes,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,0,0,1,1,1,40,1,0,0,...,1,0,5,18,15,1,0,9,4,3
1,1,0,0,0,0,0,25,1,0,0,...,0,1,3,0,0,0,0,7,6,1
2,1,0,0,1,1,1,28,0,0,0,...,1,1,5,30,30,1,0,9,4,8
3,1,0,0,1,0,1,27,0,0,0,...,1,0,2,0,0,0,0,11,3,6
4,1,0,0,1,1,1,24,0,0,0,...,1,0,2,3,0,0,0,11,5,4


### 1.3 Dataset Training and Testing Split

In [17]:
targets = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for col in targets:
    columns.remove(col)
    
# Splitting dataset
from sklearn.model_selection import train_test_split as tts

train, test = tts(df_target, test_size=.20, random_state=42, shuffle=True)

X_test  = test[columns].to_numpy()
X_train = train[columns].to_numpy()
y_test = test[targets].to_numpy()
y_train = train[targets].to_numpy()

With a large dataset of over 250,000 features, an 80/20 split is sufficient. Our classifier has plenty of data to use for training and also has plenty of data to use for training. We could comformtably move our split to 75/25 or 70/30 if we desired more opportunities to test our classifier. With such a large dataset, we could also divide the dataset with an additional validation set. With a validation set, we can use this set for more frequent model evaulations and save the testing dataset for a final unbiased evaluation.

## 2. Modeling
### Binary Logistic Regression

In [18]:
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta=0.01, iterations=50, optimization='sd', regularization='none', C=0):
        
        self.eta = eta
        self.iters = iterations
        self.opt = optimization
        self.reg = regularization
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) # return the actual prediction

In [19]:
from scipy.special import expit
from numpy.linalg import pinv

# inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    # private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # creating ability to choose optimization technique
    def _get_gradient(self,X,y):
        
        gradient = None
        if self.opt == 'sd': 
            gradient = self.steepest_descent
        elif self.opt == 'sgd': 
            gradient = self.stochastic_gradient_descent
        elif self.opt == 'newton': 
            gradient = self.newton
        else:
            print('No optimization chosen')
        return gradient(X,y)
    
    def steepest_descent(self,X,y):
    
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def stochastic_gradient_descent(self,X,y):
        
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def newton(self,X,y):
        
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
#         print('newton')
        return pinv(hessian) @ gradient
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # regularization methods
    def _get_reg_gradient(self):
        
        if self.reg == 'none':
            return self.w_[1:]
        elif self.reg == 'L1':
            return np.sign(self.w_[1:])
        elif self.reg == 'L2':
            return -2 * self.w_[1:]
        elif self.reg == 'L1_L2':
            return -2 * self.w_[1:] + np.sign(self.w_[1:])    
        
    def fit(self, X, y):
        
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        for _ in range(int(self.iters)):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate

### Logistic Regression Class

In [20]:
class LogisticRegression:
    
    def __init__(self, eta, iterations, optimization, regularization, C=0):
        
        self.eta = eta
        self.iters = iterations
        self.opt = optimization
        self.reg = regularization
        self.C = C
        self.encodings = {}

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
    
    def fit(self,X,y):
        
        self.classifiers_ = []
        for i in range(len(y[0,:])):
            blr = BinaryLogisticRegression(self.eta, self.iters, self.opt, self.reg, self.C)
            blr.fit(X,y[:,i])
            self.classifiers_.append(blr)
            
        # saving weights
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
    
    def predict_proba(self,X):
        
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
    
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

### Testing the logistic regression

In [21]:
# Validating training dataset
from sklearn.metrics import accuracy_score
lr = LogisticRegression(optimization='sd',eta=0.9, regularization='none', iterations=10, C=0.01)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.14583333333333334  -  NoDiabetes
Accuracy of Testing Dataset:  0.6964285714285714  -  PreDiabetes
Accuracy of Testing Dataset:  0.5625  -  Diabetes


### Increasing Cost

In [22]:
from sklearn.metrics import accuracy_score
lr = LogisticRegression(optimization='newton',eta=0.9, regularization='none', iterations=10, C=0.99)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.22321428571428573  -  NoDiabetes
Accuracy of Testing Dataset:  0.44642857142857145  -  PreDiabetes
Accuracy of Testing Dataset:  0.35714285714285715  -  Diabetes


### Changing Optimization Technique

In [9]:
from sklearn.metrics import accuracy_score
lr = LogisticRegression(optimization='sgd',eta=0.9, regularization='none', iterations=10, C=0.01)
for i,col in zip(range(len(y_test[0,:])), targets):
    lr.fit(X_train, y_train)
    yhat = lr.predict(X_test)
    print("Accuracy of Testing Dataset: ", accuracy_score(y_test[:,i],yhat), " - ", col)

Accuracy of Testing Dataset:  0.15651608325449384  -  NoDiabetes
Accuracy of Testing Dataset:  0.9813938820561338  -  PreDiabetes
Accuracy of Testing Dataset:  0.13818590350047305  -  Diabetes


In [ ]:
#what can i change
#C
#iterations
#

## 3. Deployment